https://www.acmicpc.net/problem/1007

### input
2
4
5 5
5 -5
-5 5
-5 -5
2
-100000 -100000
100000 100000

### output
0.000000000000
282842.712474619038

## 첫번째 시도
### 접근 방법
1. 주어진 점들을 두개씩 짝지어서 벡터를 생성.
2. 이때 한 쌍에서는 두 방향의 벡터를 만들 수 있음.
3. 첫 재귀함수에서는 중복 없이 두개씩 매칭
4. 이렇게 만든 경우의 수에서 두 방향의 벡터로 만들어지는 모든 경우의 수 탐색

역시나 시간 초과

In [2]:
from OJUtils.input import testcase

In [9]:
import math

@testcase
def main(*args, **kwargs):
    input = kwargs['testcase'].readline

    def sol():
        N = int(input())
        coordinates = [tuple(map(int, input().split())) for _ in range(N)]

        def calculate_min_length(remains: list, vector_sum: tuple = (0, 0)) -> float:
            """
            재귀함수
            선택한 두점으로 만든 두 방향의 벡터들로 만들 수 있는
            모든 벡터합 중 길이가 최소인 것을 탐색
            """

            if not remains:
                return math.sqrt(vector_sum[0]**2 + vector_sum[1]**2)

            result = 987654321
            sign = [1, -1]
            nonlocal coordinates

            remain = remains[:]
            a, b = remain.pop(0)
            xa, ya = coordinates[a]
            xb, yb = coordinates[b]
            for s in sign:
                x = s * (xa-xb)
                y = s * (ya-yb)
                result = min(result, calculate_min_length(remain,(vector_sum[0] + x, vector_sum[1] + y)))
                if math.isclose(result, 0, rel_tol = 1e-6):
                    break

            return result


        def point_paring(remains: list, selected_points: list = []) -> float:
            """
            재귀함수
            벡터로 만들 두 점을 중복된 경우의 수 없이 선택
            """
            
            if len(remains) % 2 == 1:
                return

            if not remains:
                return calculate_min_length(selected_points[:])

            points = selected_points[:]
            remain = remains[:]
            a = remain[0]
            result = 987654321

            for b in remain[1:]:
                points.append((a,b))
                remain.remove(a)
                remain.remove(b)
                result = min(result, point_paring(remain, points))
                points.pop()
                remain = remains[:]

            return result
            
        result = point_paring(list(range(N)))
        return result

    T = int(input())

    for _ in range(T):
        print(sol())

main()

0.0
282842.71247461904

걸린 시간: 0.000522sec


## 두번째 시도
### 접근 방법
1. 위 방법을 하다 보니 결국 벡터의 합을 구하므로 전체 단위로 본다면 주어진 점의 절반은 더하고, 절반은 뺀 것이 벡터의 합
2. itertools.combinations를 사용해 절반으로 분할한 뒤 반은 더하고 다른 반은 빼면 모든 경우의 벡터의 합의 길이를 구할 수 있음.
3. 이때 절반은 중복이므로 생략가능 or combinations 말고 중복 없이 고르도록 직접 함수를 제작하는 방법도 있음.
4. N이 20일때 모든 조합의 경우의 수는 184,756 가지
5. 생각해보니 combinations는 iterator를 반환하니까 굳이 절반만 연산하는 애들 만들 필요가 없음
6. 그냥 개수만 계산해서 도달하면 멈추면 됨.


In [35]:
import math
import itertools

@testcase
def main(*args, **kwargs):
    input = kwargs['testcase'].readline

    def sol() -> float:
        N = int(input())
        coordinates = [tuple(map(int, input().split())) for _ in range(N)]

        limit = math.comb(N, N//2) // 2
        case = itertools.combinations(range(N), N//2)

        result = float('inf')
        cnt = 1

        def vector_length(selected_indices: list) -> float:
            x = y = 0

            for i in range(N):
                if i in selected_indices:
                    x += coordinates[i][0]
                    y += coordinates[i][1]
                else:
                    x -= coordinates[i][0]
                    y -= coordinates[i][1]

            length = math.dist((0,0), (x,y))
            return length

        for i in case:
            if cnt > limit:
                break

            result = min(result, vector_length(i))
            cnt += 1

        return result

        T = int(input())

        for _ in range(T):
            print(sol())

main()

0.0
282842.71247461904

걸린 시간: 0.001047sec


### 다른사람의 풀이
## 접근 방법
1. 위의 방법으로는 경우의 수를 찾고나서 또 계산하는데 다시 리스트를 순회하여 각각 합한뒤 계산을 한다.
2. 벡터 합은 벡터AB + 벡터CD = (B-A) + (D-C) = A + B + C + D - 2(A+B) 로 표현 할 수 있다.
3. 즉 경우의 수를 구하면서 선택한 점의 좌표만 계속 누적해서 넘겨주면 전체 합과 누적 합으로 길이를 계산할 수 있다.

In [18]:
import math

@testcase
def solve(*args, **kwargs) -> None:
    input = kwargs['testcase'].readline

    T = int(input())
    inf = float('inf')
    x = [0] * 20
    y = [0] * 20
    
    def min_length(cnt: int, index: int, selected_x: int, selected_y: int) -> None:
        nonlocal result

        # base case
        # 필요한 만큼 수를 선택했다면 길이 계산
        if cnt == 0:
            result = min(math.sqrt((x_total - 2*selected_x)**2 + (y_total - 2*selected_y)**2), result)
            return

        # 아직 선택할게 남았다면 점을 선택
        # 중복을 제거하기 위해 사전순으로 선택
        # 점을 선택할 수 있는 범위는 이전에 선택한 값의 다음 인덱스 부터
        # 선택할 수 있는 점의 개수가 cnt 보다 크거나 같아야 하기 때문에 N - cnt 인덱스까지
        for i in range(index, N - cnt + 1):
            min_length(cnt - 1, index + 1, selected_x + x[i], selected_y + y[i])

    
    for _ in range(T):
        N = int(input())
        result = inf
        x_total = 0
        y_total = 0

        for i in range(N):
            x[i], y[i] = map(int, input().split())
            x_total += x[i]
            y_total += y[i]

        # 시작이 0이 아닌 이유
        # 모든 조합은 이 문제에서 절반이 중복이기 때문에
        # 첫번째 원소를 먼저 넣어 주면 자연스레 첫번째 원소가 들어있지 않는 나머지 절반이 제거됨.
        min_length(N//2 - 1, 1, x[0], y[0])
        print(result)

if __name__ == '__main__':
    solve()

        

0.0
282842.71247461904

걸린 시간: 0.001034sec
